In [31]:
from transformers import AutoTokenizer, AutoModel

checkpoint = "llm-research/meta-llama-3-8b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
print('vocabulary size:', len(tokenizer))

new_tokens = ['[5]', '[4]', '[3]', '[2]', '[1]']
# check if the tokens are already in the vocabulary
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
new_tokens = list(new_tokens)
print(new_tokens)

num_added_toks = tokenizer.add_tokens(new_tokens, special_tokens=True)

print("After we add", num_added_toks, "tokens")
print('vocabulary size:', len(tokenizer))

# add new, random embeddings for the new tokens
model.resize_token_embeddings(len(tokenizer))

print(model.embed_tokens.weight.size())

# Randomly generated matrix
print(model.embed_tokens.weight[-2:, :])

# save new tokenizer
tokenizer.save_pretrained("new_token")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

vocabulary size: 128256
['[3]', '[2]', '[1]', '[4]', '[5]']
After we add 5 tokens
vocabulary size: 128261
torch.Size([128261, 4096])
tensor([[ 0.0027,  0.0267,  0.0343,  ...,  0.0065, -0.0214,  0.0078],
        [-0.0338, -0.0048, -0.0302,  ...,  0.0280,  0.0134,  0.0313]],
       grad_fn=<SliceBackward0>)


('new_token/tokenizer_config.json',
 'new_token/special_tokens_map.json',
 'new_token/tokenizer.json')

In [33]:
tokenizer = AutoTokenizer.from_pretrained("new_token")

print('vocabulary size:', len(tokenizer))

vocabulary size: 128261


在使用预训练模型时，我们有时需要使用一些自定义 token 来增强输入，例如使用 `[ENT_START]` 和 `[ENT_END]` 在文本中标记出实体。由于自定义 token 并不在预训练模型原来的词表中，因此直接运用分词器 (Tokenizer) 处理输入就会出现问题。

例如直接使用 BERT 分词器处理下面的句子：

In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")

sentence = 'Two [ENT_START] cars [ENT_END] collided in a [ENT_START] tunnel [ENT_END] this morning.'
print(tokenizer(sentence).tokens())

['<|begin_of_text|>', 'Two', 'Ġ[', 'ENT', '_START', ']', 'Ġcars', 'Ġ[', 'ENT', '_END', ']', 'Ġcollided', 'Ġin', 'Ġa', 'Ġ[', 'ENT', '_START', ']', 'Ġtunnel', 'Ġ[', 'ENT', '_END', ']', 'Ġthis', 'Ġmorning', '.']


由于分词器无法识别 `[ENT_START]` 和 `[ENT_END]` ，将它们都当作未知字符处理，例如 `[ENT_END]` 被切分成了 `'['、'E'、'##NT'、'_'、'E'、'##ND'、']'` 七个 token，很明显不符合我们的预期。

此外，有时我们还会遇到一些领域相关词汇，例如医学领域的文本通常会包含大量的医学术语，它们可能并不在模型的词表中（例如一些术语是使用多个词语的缩写拼接而成），这时也会出现上面的问题。

此时我们就需要将这些新 token 添加到模型的词表中，让分词器与模型可以识别并处理这些 token。

## 添加新 token
### 添加方法

Huggingface 的 Transformers 库提供了两种方式来添加新 token，分别是：

- `add_tokens()` 添加普通 token：添加新 token 列表，如果 token 不在词表中，就会被添加到词表的最后。

In [7]:
tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")
  
num_added_toks = tokenizer.add_tokens(["new_tok1", "my_new-tok2"])
print("We have added", num_added_toks, "tokens")

We have added 2 tokens


为了防止 token 已经包含在词表中，我们还可以预先对新 token 列表进行过滤：

In [8]:
new_tokens = ["new_tok1", "my_new-tok2"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))

0

- `add_special_tokens()` 添加特殊 token：添加包含特殊 token 的字典，键值从 bos_token, eos_token, unk_token, sep_token, pad_token, cls_token, mask_token, additional_special_tokens 中选择。与 add_tokens() 类似，如果 token 不在词表中，就会被添加到词表的最后。添加后，还可以通过特殊属性来访问这些 token，例如 tokenizer.cls_token 就指向 cls token。

In [9]:
tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")

special_tokens_dict = {"cls_token": "[MY_CLS]"}
  
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("We have added", num_added_toks, "tokens")
  
assert tokenizer.cls_token == "[MY_CLS]"

We have added 1 tokens


我们也可以使用 `add_tokens()` 添加特殊 token，只需要额外设置参数 `special_tokens=True`：

In [10]:
tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")
  
num_added_toks = tokenizer.add_tokens(["[NEW_tok1]", "[NEW_tok2]"])
num_added_toks = tokenizer.add_tokens(["[NEW_tok3]", "[NEW_tok4]"], special_tokens=True)
  
print("We have added", num_added_toks, "tokens")
print(tokenizer('[NEW_tok1] Hello [NEW_tok2] [NEW_tok3] World [NEW_tok4]!').tokens())

We have added 2 tokens
['<|begin_of_text|>', '[NEW_tok1]', 'ĠHello', 'Ġ', '[NEW_tok2]', 'Ġ', '[NEW_tok3]', 'ĠWorld', 'Ġ', '[NEW_tok4]', '!']


特殊 token 的标准化 (normalization) 过程与普通 token 有一些不同，比如不会被小写。

对于之前的例子，很明显实体标记符 `[ENT_START]` 和 `[ENT_END]` 属于特殊 token，因此按添加特殊 token 的方式进行。如果使用 `add_tokens()` 则需要额外设置 `special_tokens=True`，或者也可以直接使用 `add_special_tokens()`。

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")

num_added_toks = tokenizer.add_tokens(['[ENT_START]', '[ENT_END]'], special_tokens=True)
# num_added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[ENT_START]', '[ENT_END]']})
print("We have added", num_added_toks, "tokens")

sentence = 'Two [ENT_START] cars [ENT_END] collided in a [ENT_START] tunnel [ENT_END] this morning.'

print(tokenizer(sentence).tokens())

We have added 2 tokens
['<|begin_of_text|>', 'Two', 'Ġ', '[ENT_START]', 'Ġcars', 'Ġ', '[ENT_END]', 'Ġcollided', 'Ġin', 'Ġa', 'Ġ', '[ENT_START]', 'Ġtunnel', 'Ġ', '[ENT_END]', 'Ġthis', 'Ġmorning', '.']


可以看到，分词器成功地将 `[ENT_START]` 和 `[ENT_END]` 识别为 token，并且依旧保持大写。

### 调整 embedding 矩阵

**注意！无论使用哪种方式向词表中添加新 token 后，都需要重置模型 token embedding 矩阵的大小，也就是向矩阵中添加新 token 对应的 embedding，这样模型才可以正常工作（将 token 映射到对应的 embedding）。**

该操作通过调用预训练模型的 `resize_token_embeddings()` 函数来实现，例如对于上面的例子：

In [22]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("llm-research/meta-llama-3-8b")
model = AutoModel.from_pretrained("llm-research/meta-llama-3-8b")

print(len(tokenizer))
num_added_toks = tokenizer.add_tokens(['[ENT_START]', '[ENT_END]'], special_tokens=True)
print("We have added", num_added_toks, "tokens")
print(len(tokenizer))

model.resize_token_embeddings(len(tokenizer))
print(model.embed_tokens.weight.size())

# Randomly generated matrix
print(model.embed_tokens.weight[-2:, :])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

128256
We have added 2 tokens
128258
torch.Size([128258, 4096])
tensor([[-0.0065, -0.0182,  0.0124,  ..., -0.0061, -0.0275, -0.0052],
        [ 0.0128,  0.0129, -0.0050,  ...,  0.0162,  0.0117, -0.0353]],
       grad_fn=<SliceBackward0>)


可以看到，在添加了特殊 token `[ENT_START]` 和 `[ENT_END]` 之后，分词器的词表大小从 128256 增加到了 128258，并且模型的 token embedding 矩阵大小也成功调整为了 
128258×4096。

我们还尝试打印出新添加 token 对应的 embedding。因为新 token 会添加在词表的末尾，因此只需打印出矩阵最后两行。如果你重复运行一下上面的代码，就会发现每次打印出的 `[ENT_START]` 和 `[ENT_END]` 的 embedding 是不同的。这是因为在默认情况下，这些新 token 的 embedding 是随机初始化的。

如果有充分的训练语料对模型进行微调或者继续预训练，那么将新添加 token 初始化为随机向量没什么问题。但是如果训练语料较少，甚至是只有很少语料的 few-shot learning 场景下，这种做法就可能存在问题。研究表明，在训练数据不够多的情况下，这些新添加 token 的 embedding 只会在初始值附近小幅波动。换句话说，即使经过训练，它们的值事实上还是随机的。

因此，在很多情况下，我们需要手工初始化这些新 token 的 embedding。对于 Transformers 库来说，可以通过直接对 embedding 矩阵赋值来实现。例如对于上面的例子，我们将这两个新 token 的 embedding 都初始化为全零向量：

In [24]:
import torch

with torch.no_grad():
    model.embed_tokens.weight[-2:, :] = torch.zeros([2, model.config.hidden_size], requires_grad=True)
print(model.embed_tokens.weight[-2:, :])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<SliceBackward0>)


注意，初始化 embedding 的过程并不可导，因此这里通过 `torch.no_grad()` 暂停梯度的计算。

还有另外一些比较常见的操作是根据新添加 token 的语义，将其值初始化为训练好 token 的 embedding。例如对于上面的例子，我们可以将 `[ENT_START]` 和 `[ENT_END]` 的值都初始化为“entity”对应的 embedding。因为 token id 就是 token 在矩阵中的索引，因此我们可以直接通过 `weight[token_id]` 取出“entity”对应的 embedding。

In [25]:
token_id = tokenizer.convert_tokens_to_ids('entity')
token_embedding = model.embed_tokens.weight[token_id]
print(token_id)

with torch.no_grad():
    for i in range(1, num_added_toks+1):
        model.embed_tokens.weight[-i:, :] = token_embedding.clone().detach().requires_grad_(True)
print(model.embed_tokens.weight[-2:, :])

3069
tensor([[-0.0030,  0.0055,  0.0151,  ..., -0.0129,  0.0041,  0.0093],
        [-0.0030,  0.0055,  0.0151,  ..., -0.0129,  0.0041,  0.0093]],
       grad_fn=<SliceBackward0>)


可以看到最终结果符合我们的预期，`[ENT_START]` 和 `[ENT_END]` 被初始化为相同的 embedding。

另一种常见的做法是根据新 token 的语义，使用对应的描述文本来完成初始化。例如将值初始化为描述文本中所有 token 的平均值，假设新 token $t_i$ 的描述文本为 $w_{i,1}, w_{i,2}, \ldots, w_{i,n}$，那么 $t_i$ 的初始化 embedding 为：

$E(t_i) = \frac{1}{n} \sum_{j=1}^{n} E(w_{i,j})$

这里 $E$ 表示预训练模型的 token embedding 矩阵。对于上面的例子，我们可以分别为 `[ENT_START]` 和 `[ENT_END]` 编写对应的描述，然后再到它们的值进行初始化：

In [27]:
descriptions = ['start of entity', 'end of entity']

with torch.no_grad():
    for i, token in enumerate(reversed(descriptions), start=1):
        tokenized = tokenizer.tokenize(token)
        print(tokenized)
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized)
        new_embedding = model.embed_tokens.weight[tokenized_ids].mean(axis=0)
        model.embed_tokens.weight[-i, :] = new_embedding.clone().detach().requires_grad_(True)

print(model.embed_tokens.weight[-2:, :])

['end', 'Ġof', 'Ġentity']
['start', 'Ġof', 'Ġentity']
tensor([[-0.0044, -0.0003,  0.0006,  ..., -0.0017, -0.0066, -0.0005],
        [-0.0054, -0.0013,  0.0025,  ...,  0.0010, -0.0039,  0.0031]],
       grad_fn=<SliceBackward0>)


可以看到，这里成功地将 `[ENT_START]` 初始化为“start”、“of”、“entity”三个 token embedding 的平均值，将 `[ENT_END]` 初始化为“end”、“of”、“entity” embedding 的平均值。

参考：
1. https://xiaosheng.blog/2023/01/07/add-new-token
2. https://stackoverflow.com/questions/76198051/how-to-add-new-tokens-to-an-existing-huggingface-tokenizer